In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
data_test= pd.read_csv('data_for_testing.csv', delimiter=";")
data_from_users= pd.read_csv('data_from_users.csv', delimiter=";")
users_featured= pd.read_csv('users_featured.csv', delimiter=";")

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
sampler = SMOTE()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
#Logistic Regression Classifier Confusion matrix
from sklearn.metrics import confusion_matrix

In [5]:
from sklearn.model_selection import train_test_split

X = data_test.drop(columns=['plan']).values
X_columns = data_test.drop(columns=['plan']).columns
y = data_test['plan'].values

data_test.plan.value_counts()

0    14394
1     1150
Name: plan, dtype: int64

In [6]:
# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
print(X.shape)
print(y.shape)

(28788, 51)
(28788,)


In [7]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, stratify=y)
X_ts.shape

(5758, 51)

In [8]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, stratify=y)
y_ts.shape

(5758,)

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
sc = StandardScaler()
from sklearn.feature_selection import VarianceThreshold
var = VarianceThreshold()
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
sampler = SMOTE()
rf = RandomForestClassifier()
#Logistic Regression Classifier Confusion matrix
from sklearn.metrics import confusion_matrix

Feature Selection

In [10]:
X.shape

(28788, 51)

In [11]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

(28788, 38)

In [12]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X_new, y, test_size=0.2, stratify=y)
X_ts.shape

(5758, 38)

Training Model

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern

param_grid = [
    { 
        'preprocessing__truncate_titles__n_values_to_keep': [5],
        'classifier': [ GaussianProcessClassifier() ], 
        'classifier__kernel': [1.0*RBF(1.0), 1.0*Matern(1.0)]
    }
]

gp_grid_search = build_grid_search(pipeline=build_pipeline(), param_grid=param_grid)
gp_cv_results = run_grid_search(grid_search=gp_grid_search)

NameError: name 'build_grid_search' is not defined

In [24]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
pipe_nb=  Pipeline([('scaler', sc),
                  ('selector', var),
                  ('sampler', sampler),
                  ('model', nb)])
grid_nb= {'selector__threshold': [0.1, 0.2, 0.3],
        }
clf_nb= GridSearchCV(pipe_nb, grid_nb, cv=3)
clf_nb.fit(X_tr, y_tr)

nb_preds = clf_nb.predict(X_ts)

In [26]:
from sklearn.metrics import classification_report , accuracy_score , precision_recall_fscore_support
print(clf_nb.best_params_, '\n')
print('accuracy: {:.2f}%'.format(accuracy_score(y_ts, nb_preds)*100))

print(classification_report(y_ts, nb_preds))
confusion_matrix(y_ts,nb_preds)

{'selector__threshold': 0.2} 

accuracy: 87.15%
              precision    recall  f1-score   support

           0       0.83      0.94      0.88      2879
           1       0.93      0.80      0.86      2879

    accuracy                           0.87      5758
   macro avg       0.88      0.87      0.87      5758
weighted avg       0.88      0.87      0.87      5758



array([[2707,  172],
       [ 568, 2311]], dtype=int64)